In [22]:
from bs4 import BeautifulSoup
import requests
import psycopg2

conn = psycopg2.connect(database='solidot', user='python', password='passw0rd3pyth4n', host='127.0.0.1', port='5432')
db = conn.cursor()
db.execute('''CREATE TABLE IF NOT EXISTS guazi \
           (id INT PRIMARY KEY    NOT NULL,
           prise     TEXT         NOT NULL,
           context   TEXT);''')
conn.commit()

head = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.19 Safari/537.36'}
for page in range(1,30):
    if page == 1:
        html = 'http://www.guazi.com/sjz/buy/'
    else:
        html = 'http://www.guazi.com/sjz/buy/o' + str(page)
    res = requests.get(html, headers=head)
    soup = BeautifulSoup(res.text, 'html.parser')
    titles = soup.select('div.list > ul > li > div > p.infoBox > a')
    prices = soup.select('div.list > ul > li > div > p.priType-s > span > i')
    times = len(soup.select('div.list > ul > li > div > p.infoBox > a'))
    for i in range(times):
        prise = prices[i].text.strip()
        context = titles[i].text
        db.execute("INSERT INTO guazi VALUES (%s, %s, %s)", (i + page * times + 1, prise, context))   
        
conn.commit()
conn.close()
print("OK")


OK
